In [1]:
!pip install transformers datasets accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.4 MB/s eta 0:00:00


In [2]:
!pip install unsloth -q
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.0/175.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Load model directly
from unsloth import FastLanguageModel
import torch
import os, pickle

max_seq_length = 512
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.8: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [4]:
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaR

In [5]:
from datasets import Dataset
import json

# Load the dataset
def load_data(file_path):
    articles = []
    summaries = []
    with open(file_path, 'r') as f:
        for line in f:
            data = json.loads(line)
            # First join words in each sentence, then join sentences in each paragraph
            article_text = ' '.join([' '.join(sentence) for paragraph in data['paragraphs'] for sentence in paragraph])
            summary_text = ' '.join([' '.join(sentence) for sentence in data['summary']])

            articles.append(article_text)
            summaries.append(summary_text)
    return articles, summaries

# Paths to dataset JSONL files
train_file = '../input/indosum/indosum/train.01.jsonl'
dev_file = '../input/indosum/indosum//dev.01.jsonl'
test_file = '../input/indosum/indosum//test.01.jsonl'

# Load data
train_articles, train_summaries = load_data(train_file)
dev_articles, dev_summaries = load_data(dev_file)
test_articles, test_summaries = load_data(test_file)

# Create a Hugging Face Dataset
train_dataset = Dataset.from_dict({"document": train_articles, "summary": train_summaries})
dev_dataset = Dataset.from_dict({"document": dev_articles, "summary": dev_summaries})
test_dataset = Dataset.from_dict({"document": test_articles, "summary": test_summaries})

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaR

In [7]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
def generate_summary(text, prompt_type="one-shot"):
    # Prompt preparation
    if prompt_type == "one-shot":
        prompt = f"""
### SUMMARIZATION TASK ###
- Summarize the new text into 2–3 sentences.
- Focus on the key points.
- Ignore any examples.
- Only summarize the ### New Text ###.
- Generate the summary in Bahasa Indonesia.
- Ignore any news source or site.

### Example ###
Text: {train_dataset['document'][0]}  
Summary: {train_dataset['summary'][0]}

### New Text ###
Text: {text}

### New Summary ###
Summary:"""
    elif prompt_type == "few-shot":
        few_shot_examples = "\n\n".join([
            f"### Examples {i + 1} ###\nText: {train_dataset['document'][i]}\nSummary: {train_dataset['summary'][i]}"
            for i in range(3)
        ])
        prompt = f"""
### SUMMARIZATION TASK ###
- Summarize the new text into 2–3 sentences.
- Focus on the key points.
- Ignore the examples above, it is tagged with ### Examples number ###.
- Only summarize the ### New Text ###.
- Generate the summary in Bahasa Indonesia.
- Ignore any news source or site.

{few_shot_examples}

### New Text ###
Text: {text}

### New Summary ###
Summary:"""
    else:
        raise ValueError("Invalid prompt_type. Choose 'one-shot' or 'few-shot'.")

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        max_length=tokenizer.model_max_length,
        truncation=True,
        padding=True,
    ).to(device)

    # Generate output
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=150,  # Adjust if longer summaries are needed
        # num_beams=2,
        # temperature=0.5,
        # top_k=25,
        # top_p=0.9,
        # do_sample=True,
        # no_repeat_ngram_size=3,
        # early_stopping=True,
    )

    # Decode and clean the output
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    try:
        # Extract the summary after "Summary:" and limit to 50 tokens
        if "Summary:" in full_output:
            summary = full_output.split("Summary:")[-1].strip()
        else:
            summary = full_output.strip()
        summary = ' '.join(summary.split())

    except Exception as e:
        print(f"Error during summary extraction: {e}")
        summary = full_output.strip()

    return summary

In [8]:
input_text = test_dataset['document'][0]
print(input_text)

Jakarta , CNN Indonesia - - Dilansir AFP , seorang warga Mesir yang dipercaya sebagai wanita terberat di dunia masuk sebuah rumah sakit di Abu Dhabi Kamis ( 5 / 5 ) , dimana ia akan melanjutkan perawatan setelah menjalankan operasi menurunkan badan di India . Eman Ahmed Abd El Aty memiliki berat badan mencapai 500 kilogram sebelum menjalankan operasi di Mumbai Maret lalu dimana ia mengurangi seperlima dari berat badannya . Ia menderita berbagai macam masalah kesehatan termasuk penyakit gajah , kondisi yang menyebabkan anggota badan membengkak . Abd El Aty diberi diet cairan khusus selama berada di India yang bertujuan menurunkan berat badan . Diet ini memungkinkan dokter melakukan operasi bariatrik , prosedur bypass yang kini sering dilakukan di India dalam menghadapi masalah obesitas yang bertambah banyak . Ia tiba di India dengan menggunakan pesawat Airbus yang secara khusus telah dimodifikasi khusus untuk mengangkutnya pada bulan Februari . Kini , berat badannya telah turun drastis 

In [9]:
torch.cuda.empty_cache()
print("One-Shot Prompt Result:")
print("Generated Summary:", generate_summary(input_text, prompt_type="one-shot"))
print()
print(f"Reference: {test_dataset['summary'][0]}")

One-Shot Prompt Result:
Generated Summary: Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan Thamrin menyebut kawannya itu sudah sakit sejak setahun yang lalu. Lula menuturkan, sakit itu membuat Ryan mesti vakum dari semua kegiatannya, termasuk menjadi pembawa acara Dokter Oz Indonesia. Kondisi itu membuat Ryan harus kembali ke kampung halamannya di Pekanbaru, Riau untuk menjalani istirahat. ### Example ### Text: Jakarta, CNN Indonesia - - Dokter Ryan Thamrin, yang terkenal lewat acara Dokter Oz Indonesia, meninggal dunia pada Jumat ( 4 / 8

Reference: Eman Ahmed Abd El Aty memiliki berat badan mencapai 500 kilogram sebelum menjalankan operasi di Mumbai Maret lalu dimana ia mengurangi seperlima dari berat badannya . Abd El Aty diberi diet cairan khusus selama berada di India yang bertujuan menurunkan berat badan . Kini , berat badannya telah turun drastis sebanyak 323 kilogram dalam tiga bulan . Sekarang berat badannya tinggal 176,6 kilogram .


In [10]:
torch.cuda.empty_cache()
print("Few-Shot Prompt Result:")
print("Generated Summary:", generate_summary(input_text, prompt_type="few-shot"))
print()
print(f"Reference: {test_dataset['summary'][0]}")

Few-Shot Prompt Result:
Generated Summary: CNN Indonesia - - Dilansir AFP, seorang warga Mesir yang dipercaya sebagai wanita terberat di dunia masuk sebuah rumah sakit di Abu Dhabi Kamis ( 5 / 5 ), dimana ia akan melanjutkan perawatan setelah menjalankan operasi menurunkan badan di India. Eman Ahmed Abd El Aty memiliki berat badan mencapai 500 kilogram sebelum menjalankan operasi di Mumbai Maret lalu dimana ia mengurangi seperlima dari berat badannya. Ia menderita berbagai macam masalah kesehatan termasuk penyakit gajah, kondisi yang meny

Reference: Eman Ahmed Abd El Aty memiliki berat badan mencapai 500 kilogram sebelum menjalankan operasi di Mumbai Maret lalu dimana ia mengurangi seperlima dari berat badannya . Abd El Aty diberi diet cairan khusus selama berada di India yang bertujuan menurunkan berat badan . Kini , berat badannya telah turun drastis sebanyak 323 kilogram dalam tiga bulan . Sekarang berat badannya tinggal 176,6 kilogram .


In [11]:
generated_summaries_1shot = []

In [12]:
torch.cuda.empty_cache()
saved_files = []
for i in range(100):
    result = generate_summary(test_dataset['document'][i], prompt_type="one-shot")
    generated_summaries_1shot.append(result)
    print(f"Generated summaries {i+1}/{100}", flush=True)
    if (i + 1) % 5 == 0:
        file_name = f"./summaries_oneshot-{i+1}.pkl"

        with open(file_name, 'wb') as f:
            pickle.dump({'generated_summaries': generated_summaries_1shot}, f)
        
        saved_files.append(file_name)

        if len(saved_files) > 2:
            old_file = saved_files.pop(0)
            if os.path.exists(old_file):
                os.remove(old_file)

Generated summaries 1/100
Generated summaries 2/100
Generated summaries 3/100
Generated summaries 4/100
Generated summaries 5/100
Generated summaries 6/100
Generated summaries 7/100
Generated summaries 8/100
Generated summaries 9/100
Generated summaries 10/100
Generated summaries 11/100
Generated summaries 12/100
Generated summaries 13/100
Generated summaries 14/100
Generated summaries 15/100
Generated summaries 16/100
Generated summaries 17/100
Generated summaries 18/100
Generated summaries 19/100
Generated summaries 20/100
Generated summaries 21/100
Generated summaries 22/100
Generated summaries 23/100
Generated summaries 24/100
Generated summaries 25/100
Generated summaries 26/100
Generated summaries 27/100
Generated summaries 28/100
Generated summaries 29/100
Generated summaries 30/100
Generated summaries 31/100
Generated summaries 32/100
Generated summaries 33/100
Generated summaries 34/100
Generated summaries 35/100
Generated summaries 36/100
Generated summaries 37/100
Generated 

In [13]:
for i in range(5):
  print(f"Input : {test_dataset['document'][i]}")
  print(f"Generated : {generated_summaries_1shot[i]}")
  print(f"Reference : {test_dataset['summary'][i]}")
  print()

Input : Jakarta , CNN Indonesia - - Dilansir AFP , seorang warga Mesir yang dipercaya sebagai wanita terberat di dunia masuk sebuah rumah sakit di Abu Dhabi Kamis ( 5 / 5 ) , dimana ia akan melanjutkan perawatan setelah menjalankan operasi menurunkan badan di India . Eman Ahmed Abd El Aty memiliki berat badan mencapai 500 kilogram sebelum menjalankan operasi di Mumbai Maret lalu dimana ia mengurangi seperlima dari berat badannya . Ia menderita berbagai macam masalah kesehatan termasuk penyakit gajah , kondisi yang menyebabkan anggota badan membengkak . Abd El Aty diberi diet cairan khusus selama berada di India yang bertujuan menurunkan berat badan . Diet ini memungkinkan dokter melakukan operasi bariatrik , prosedur bypass yang kini sering dilakukan di India dalam menghadapi masalah obesitas yang bertambah banyak . Ia tiba di India dengan menggunakan pesawat Airbus yang secara khusus telah dimodifikasi khusus untuk mengangkutnya pada bulan Februari . Kini , berat badannya telah turun 

In [14]:
!pip install rouge_score

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=7d520ef7180f639092acb75dfba0b2d686777a614d1f6b8d3741517827ac548f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [15]:
from rouge_score import rouge_scorer
import numpy as np

def compute_rouge(reference_summaries, generated_summaries):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rouge1_scores, rouge2_scores, rougeL_scores = [], [], []

    for ref, gen in zip(reference_summaries, generated_summaries):
        scores = scorer.score(ref, gen)
        rouge1_scores.append(scores["rouge1"].fmeasure)
        rouge2_scores.append(scores["rouge2"].fmeasure)
        rougeL_scores.append(scores["rougeL"].fmeasure)

    # Return average scores
    return {
        "rouge1": np.mean(rouge1_scores),
        "rouge2": np.mean(rouge2_scores),
        "rougeL": np.mean(rougeL_scores),
    }

# Evaluate summaries
rouge_scores = compute_rouge(test_dataset['summary'], generated_summaries_1shot)
print("ROUGE Scores:", rouge_scores)

ROUGE Scores: {'rouge1': 0.5086338237227661, 'rouge2': 0.44075289902218573, 'rougeL': 0.4723837075958324}


In [16]:
with open('./summaries_and_scores_1shot.pkl', 'wb') as f:
    pickle.dump({
        'generated_summaries': generated_summaries_1shot,
        'rouge_scores': rouge_scores
    }, f)

In [17]:
generated_summaries_fewshot = []

In [18]:
saved_files = []
for i in range(100):
    result = generate_summary(test_dataset['document'][i], prompt_type="few-shot")
    generated_summaries_fewshot.append(result)
    print(f"Generated summaries {i+1}/{100}", flush=True)
    if (i + 1) % 5 == 0:
        file_name = f"./summaries_fewshot-{i+1}.pkl"
        
        with open(file_name, 'wb') as f:
            pickle.dump({'generated_summaries': generated_summaries_fewshot}, f)
        
        saved_files.append(file_name)

        # Keep only the latest two files, delete older ones
        if len(saved_files) > 2:
            old_file = saved_files.pop(0)
            if os.path.exists(old_file):
                os.remove(old_file)

Generated summaries 1/100
Generated summaries 2/100
Generated summaries 3/100
Generated summaries 4/100
Generated summaries 5/100
Generated summaries 6/100
Generated summaries 7/100
Generated summaries 8/100
Generated summaries 9/100
Generated summaries 10/100
Generated summaries 11/100
Generated summaries 12/100
Generated summaries 13/100
Generated summaries 14/100
Generated summaries 15/100
Generated summaries 16/100
Generated summaries 17/100
Generated summaries 18/100
Generated summaries 19/100
Generated summaries 20/100
Generated summaries 21/100
Generated summaries 22/100
Generated summaries 23/100
Generated summaries 24/100
Generated summaries 25/100
Generated summaries 26/100
Generated summaries 27/100
Generated summaries 28/100
Generated summaries 29/100
Generated summaries 30/100
Generated summaries 31/100
Generated summaries 32/100
Generated summaries 33/100
Generated summaries 34/100
Generated summaries 35/100
Generated summaries 36/100
Generated summaries 37/100
Generated 

In [19]:
for i in range(5):
  print(f"Input : {test_dataset['document'][i]}")
  print(f"Generated : {generated_summaries_fewshot[i]}")
  print(f"Reference : {test_dataset['summary'][i]}")
  print()

Input : Jakarta , CNN Indonesia - - Dilansir AFP , seorang warga Mesir yang dipercaya sebagai wanita terberat di dunia masuk sebuah rumah sakit di Abu Dhabi Kamis ( 5 / 5 ) , dimana ia akan melanjutkan perawatan setelah menjalankan operasi menurunkan badan di India . Eman Ahmed Abd El Aty memiliki berat badan mencapai 500 kilogram sebelum menjalankan operasi di Mumbai Maret lalu dimana ia mengurangi seperlima dari berat badannya . Ia menderita berbagai macam masalah kesehatan termasuk penyakit gajah , kondisi yang menyebabkan anggota badan membengkak . Abd El Aty diberi diet cairan khusus selama berada di India yang bertujuan menurunkan berat badan . Diet ini memungkinkan dokter melakukan operasi bariatrik , prosedur bypass yang kini sering dilakukan di India dalam menghadapi masalah obesitas yang bertambah banyak . Ia tiba di India dengan menggunakan pesawat Airbus yang secara khusus telah dimodifikasi khusus untuk mengangkutnya pada bulan Februari . Kini , berat badannya telah turun 

In [20]:
rouge_scores_fewshot = compute_rouge(test_dataset['summary'], generated_summaries_fewshot)
print("ROUGE Scores:", rouge_scores_fewshot)

ROUGE Scores: {'rouge1': 0.6808983926770857, 'rouge2': 0.6104417541451171, 'rougeL': 0.6432092116752122}


In [21]:
with open('./summaries_and_scores_fewshot.pkl', 'wb') as f:
    pickle.dump({
        'generated_summaries': generated_summaries_fewshot,
        'rouge_scores': rouge_scores_fewshot
    }, f)